**Loading Packages**

In [0]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Loading Dataset**


In [0]:
!mkdir -p ~/.kaggle

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!ls

kaggle.json  sample_data


In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
sudalairajkumar/novel-corona-virus-2019-dataset             Novel Corona Virus 2019 Dataset                     329KB  2020-03-05 06:14:42          22592  
kimjihoo/coronavirusdataset                                 Coronavirus-Dataset                                  22KB  2020-03-06 10:55:10           5703  
rupals/gpu-runtime                                          Segmentation GPU Kernel Performance Dataset           4MB  2020-03-01 10:04:27             76  
shivamb/real-or-fake-fake-jobposting-prediction             [Real or Fake] Fake JobPosting Prediction            16MB  2020-02-29 08:23:34            496  
tapakah68/yandextoloka-water-meters-dataset                 Wate

In [0]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria
!ls

 98% 662M/675M [00:05<00:00, 135MB/s]
100% 675M/675M [00:05<00:00, 127MB/s]
cell-images-for-detecting-malaria.zip  kaggle.json  sample_data


In [0]:
!unzip cell-images-for-detecting-malaria.zip

In [0]:
!ls

cell_images  cell-images-for-detecting-malaria.zip  kaggle.json  sample_data


**Data Preprocessing**

In [0]:
print(os.listdir("./cell_images/cell_images"))

['Uninfected', 'Parasitized']


In [0]:
infected = os.listdir('./cell_images/cell_images/Parasitized/') 
uninfected = os.listdir('./cell_images/cell_images/Uninfected/')

In [0]:
data = []
labels = []

for i in infected:
    try:
    
        image = cv2.imread("../input/cell_images/cell_images/Parasitized/"+i)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((50 , 50)) #image resizing
        rotated45 = resize_img.rotate(45) #rotation
        rotated75 = resize_img.rotate(75)
        blur = cv2.blur(np.array(resize_img) ,(10,10)) #bluring
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        data.append(np.array(blur)) #
        labels.append(1)
        labels.append(1)
        labels.append(1)
        labels.append(1)
        
    except AttributeError:
        print('')
    
for u in uninfected:
    try:
        
        image = cv2.imread("../input/cell_images/cell_images/Uninfected/"+u)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((50 , 50))
        rotated45 = resize_img.rotate(45)
        rotated75 = resize_img.rotate(75)
        data.append(np.array(resize_img))
        data.append(np.array(rotated45))
        data.append(np.array(rotated75))
        labels.append(0)
        labels.append(0)
        labels.append(0)
        
    except AttributeError:
        print('')

data = np.array(data)
labels = np.array(labels)

np.save('Data' , data)
np.save('Labels' , labels)

In [0]:
n = np.arange(data.shape[0]) #shuffle images
np.random.shuffle(n)
data = data[n]
labels = labels[n]

In [0]:
data = data.astype(np.float32)
labels = labels.astype(np.int32)
data = data/255 #reduce weight

Train Test Split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(

**CNN Model**

Building CNN Layers

In [0]:
def ConvBlock(model, layers, filters,name):
    for i in range(layers):
        model.add(SeparableConv2D(filters, (3, 3), activation='relu',name=name))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
def FCN():
    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=(50, 50, 3)))
    ConvBlock(model, 1, 128,'block_1')
    ConvBlock(model, 1, 256,'block_2')
    ConvBlock(model, 1, 512,'block_3')
    ConvBlock(model, 1, 1024,'block_4')
    ConvBlock(model, 1, 2048,'block_4')
    model.add(Flatten())
    model.add(Dense(2048,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='sigmoid'))
    return model

model = FCN()